* Prénom & Nom : El Hadji Mamadou Dia
* Module : Cryptographie à clé publique
* Parcours : ACC
* Année universitaire : 2023/2024

## ---- DM1 CRYPTOSYSTEME DE RABIN ----

In [1]:
from random import* 
from Crypto.Util.number import getPrime, isPrime

## PARTIE 1 : FONCTION ARITHMETIQUE

## Question 1 : Coefficients de Bezout

In [2]:
def xgcd(p, q):
    u = 1
    tampon_1 = 0
    v = 0
    tampon_2 = 1
    while q :
        quotient = p // q
        reste = p - quotient * q
        r_1 = u - quotient * tampon_1
        r_2 = v - quotient * tampon_2
        p = q
        q = reste
        u = tampon_1
        tampon_1 = r_1
        v = tampon_2
        tampon_2 = r_2
    return u, v

In [3]:
xgcd(41, 32)

(-7, 9)

## Question 2 : Liste des racines carrées de x

In [4]:
def square_roots(x, p, q, n, u, v) :
    liste_x = []
    
    racine_p = pow(x, (p + 1) // 4, p) #racine carré d'un x qui dans Z/pZ 
    racine_q = pow(x, (q + 1) // 4, q) #racine carré d'un x qui dans Z/qZ
    multi_p = u * p
    multi_q = v * q 
    x_1  = (multi_p * racine_q + multi_q * racine_p) % n
    x_2  = (racine_q * multi_p - multi_q * racine_p) % n
    liste_x = [x_1, n - x_1, x_2, n - x_2] #Liste des 4 racines carrés de x 
    return liste_x

In [5]:
square_roots(24, 11, 7, 77, 2, -3)

[30, 47, 58, 19]

## PARTIE 2 : CHIFFREMENT INITIAL DE RABIN

## Question 3: Génération de clé

In [6]:
def keygen_rabin(t):
    p = getPrime(t) #Génération d'un nombre premier p aléatoire
    q = getPrime(t) #Génération d'un nombre premier q aléatoire
    while((p == q) and (p % 4 != 3) and (q % 4 != 3)): #Condition de Blum
        p = getPrime(t)
        q = getPrime(t)
    n = p * q
    u, v = xgcd(p, q)
    public_key = n #Clé publique 
    secret_key = (n, p, q, u, v) #Clé privée
    return public_key, secret_key

## Question 4 : Fonction de chiffrement et déchiffrement

In [7]:
def encrypt_rabin_1(plaintext, n):
    c = pow(plaintext, 2, n)
    return c

In [8]:
def decrypt_rabin_1(ciphertext, n, p, q, u, v) :
    m = square_roots(ciphertext, p, q, n, u, v)
    return m

## Question 5 : Test 1

In [9]:
def TEST1(t):
    public_key, secret_key = keygen_rabin(t)
    n, p, q, u, v = secret_key
    print("La clé publique est:\n", public_key)
    print("La clé sécrete est:\n", secret_key)
    message = randint(1, 10000)
    print("Le message claire avant le chiffrement est:\n", message)
    chiffre_message = encrypt_rabin_1(message, public_key)
    print("Le message chiffré est:\n", chiffre_message)
    déchiffre_message = decrypt_rabin_1(chiffre_message, n, p, q, u, v)
    print("Les messages possibles après déchiffrement sont:")
    for mes_dechi in déchiffre_message:
        print(mes_dechi)
    # Vérification si le message clair est dans la liste des racines carrées
    if message in déchiffre_message:
        print("Le message clair a été retrouvé dans la liste des racines carrées.\n Il est:\t", message)
    else:
        print("Le message clair n'a pas été retrouvé dans la liste des racines carrées.")

In [24]:
TEST1(16)

La clé publique est:
 2135001157
La clé sécrete est:
 (2135001157, 37963, 56239, -14269, 9632)
Le message claire avant le chiffrement est:
 5748
Le message chiffré est:
 33039504
Les messages possibles après déchiffrement sont:
5748
2134995409
483604909
1651396248
Le message clair a été retrouvé dans la liste des racines carrées.
 Il est:	 5748


## PARTIE 3: CHIFFREMENT AVEC PADDING

## Question 6 : Fonction de chiffrement et déchiffrement avec padding

In [11]:
def encrypt_rabin_2(m, n, l):
    mes = m * 2**l
    c = pow(mes, 2, n)
    return c

In [12]:
def decrypt_rabin_2(c, n, p, q, u, v, l):
    compteur = 0
    L = square_roots(c, p, q, n, u, v)
    for i in range(0, 4):
        if L[i] % (2**l) == 0:
            message = L[i]
            compteur += 1
    if compteur == 1 :
        m = message // (2**l)
    else :
        raise ValueError("Il y'a une erreur")
    return m

## Question 7 : Test 2

In [13]:
def TEST2(t):
    n = keygen_rabin(t)[0]
    n, p, q, u, v = keygen_rabin(t)[1]
    print("La clé publique est:\n", n)
    print("La clé sécrete est:\n", n, p, q, u, v)
    message = randint(1, 100000)
    print("Le message claire avant le chiffrement est:\n", message)
    l = int(t // 2)
    chiffre_padding_message = encrypt_rabin_2(message, n, l)
    print("Le message chiffré est:\n", chiffre_padding_message)
    déchiffre_padding_message = decrypt_rabin_2(chiffre_padding_message, n, p, q, u, v, l)
    print("Le message claire retrouver après déchiffrement est:\n", déchiffre_padding_message)

In [14]:
TEST2(128)

La clé publique est:
 61014894326668767398952345657610739402142797655800482189859820882417895542661
La clé sécrete est:
 61014894326668767398952345657610739402142797655800482189859820882417895542661 324678963772063245448745490774097883167 187923768198002200182723328703213419483 58184409545250151120474000938969275890 -100526154727467815020961492768934281463
Le message claire avant le chiffrement est:
 30840
Le message chiffré est:
 323644464759759329855410984026156363817785753600
Le message claire retrouver après déchiffrement est:
 30840


## PARTIE 4 : UNE ATTAQUE

## Question 8 : 

Nous avons trouvé une attaque de :
* Type cassage partiel (Car on a une information sur l'espace des messages de BOP qui est la table ASCII.)
* Mode Attaque à clair choisi.


Méthode utilisé :

* Premièrement nous avons recupérer la clé public pk dans un tableau et le chiffré ciphertext.
* En plus, nous avons construit un fichier texte de quelques lettres de l'alphabet français de la table ascii avec leur entier correspondant.
* En fin, nous avons procéder à l'attaque qui consiste à parcourir l'ensemble de la table ASCII et de chiffrer chaque lettre puis le comparer avec les 24 lignes de la chiffré(ciphertext). Si on trouve un chiffré d'une lettre avec le même chiffré, on le recupére dans un tableau à la même position.

Solution:

Pour éviter cette attaque, il faut que BOP utilise le chiffrement de Rabin sans padding comme ça l'attaquant n'aura pas directement le chiffré.

Probléme renconter :

Le probléme renconté est aprés le chiffré tous lettres ne respecte pas leur position pour afficher le message correctement. Mais nous savons que le message a était 

"BRAVO VOUS AVEZ TROUVE !"

In [15]:
def recup_pk(): #Fonction qui permet de recupérer la clé publique pk
    with open('pk.txt', 'r') as fichier: #Ouverture du fichier pk.txt en mode lecture
        n = fichier.read()
    return int(n.strip()) #Recupération de la contenue comme un entier

In [16]:
recup_pk()

26819615417505517037360562715277805233497643787098333114243401591045393314685826255792983860657199260109690834177725207056599397527656983678277086700595103714830083561581670415257759847019148776219275691214843361851018009985129181754134352996938581692134197883019757683516087177487600562294796910547252132168969849355781782732183809708723312202334144154295744852820527098130543531165197721596391176833439476108668822204072351358404342748005241506175093396286427782491647297774720819052815698953330425981872377627536973225250795435951241892483945979368694093354379296163191841061112897558566432601457359820541854428017

In [17]:
def recup_cipher_text(): #Fonction qui permet de recupérer le message chiffrer 
    C = []
    with open('ciphertext.txt', 'r') as fichier: #Ouverture du fichier ciphertext.txt en mode lecture
        c = fichier.readlines() #Lecture ligne par ligne
        for mot in c:
            C.append(int(mot.strip())) #Recupération des lignes dans un tableau
    return C

In [18]:
recup_cipher_text()


[23536735561794389845769726068388874298876634588596260410413132491173669437316540365430047864829330772397940011085886283351161434394601946605840790645616978039823280782532636391137723540831799604868245305500364419564388005293711995269654410468521849533378841382456443725855076097954911371404064031861569710705066826252503806003116818895322281743914288137389897384795241056857643883095485500665534494465149839916358870648150720792426877894462135175616659533026176709194463878597975173607547015434769481053493692026540718944189653073544616311079125065482431704148355682717375946286806415814919238681073991571949142504320,
 7024710865514053311551735435978980228236388096219740444402964026004518163134041423227052035441724434224430730912655505891596958824071681971780198256725405568682401797204016509959393070877381168119910271809411376202961253458609860202818350717381374519715460866038356528721982717586741364737762455471137416358258231585930740532544204783889820101691723430064720963254704288156430271

In [19]:
def recup_code_acsii(): #Fonction pour recuprérer la table de ascii 
    tableau_ascii = []
    nomb = []
    carac = []
    with open('acsii.txt', 'r', encoding='utf-8') as fichier:#Ouverture du fichier ascii.txt en mode lecture avec encoding UTF-8
        lignes = fichier.readlines()
        for mot in lignes:
            elements = mot.strip().split('_') #Recupération de chaque colonne qui est délimité par '_'
            if len(elements) == 2:
                # Ajoutez les informations dans le tableau sous forme de tuple (nombre, caractères)
                tableau_ascii.append((elements[0], elements[1]))
    for ascii in tableau_ascii:
        nombre, caracteres = ascii
        nomb.append(nombre) #Le numéro ascii pour chaque lettre
        carac.append(caracteres) #La lettre ascii 
    return nomb, carac

In [20]:
def attaque(): #Fonction pour attaquer le chiffrement en utilisant les fonctions précedentes
    car = []
    for i in range(0, 65): #Parcours la liste des 65 lettre ascii qui constitue les lettres de l'alphabet.
        c = encrypt_rabin_2(int(recup_code_acsii()[0][i]), recup_pk(), 1024) #Calculer le chiffrement de chaque lettre ascii avec son numéro correspondant
        for j in range(0, 24): #Parcourir la liste des 24 lignes dans le chiffré qui correspond chacun une lettre
            if c == recup_cipher_text()[j]: #Si on tombe sur un chiffrer qui égale à une ligne dans le chiffré
                car.insert(j, recup_code_acsii()[1][i]) #On le recupére dans un tableau
    return ''.join(car) #Au final pour trouver les 24 caractères déchiffré dans une chaine de caractère

In [21]:
attaque()

'BRVAVOUOVSZ!AEEORVTU'